In [2]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.33.1
Uninstalling transformers-4.33.1:
  Successfully uninstalled transformers-4.33.1
Found existing installation: accelerate 0.22.0
Uninstalling accelerate-0.22.0:
  Successfully uninstalled accelerate-0.22.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/13/30/54b59e73400df3de506ad8630284e9fd63f4b94f735423d55fc342181037/transformers-4.33.1-py3-none-any.whl.metadata
  Using cached transformers-4.33.1-py3-none-any.whl.metadata (119 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/4d/a7/05c67003d659a0035f2b3a8cf389c1d9645865aee84a73ce99ddab16682f/accelerate-0.22.0-py3-none-any.whl.metadata
  Using cached accelerate-0.22.0-py3-none-any.whl.metadata (17 kB)
Using cached transformers-4.33.1-py3-none-any.whl (7.6 MB)
Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)


In [3]:
import os 

In [4]:
%pwd

'd:\\Projects and Work\\Projects\\NLP\\NLP PROJECTS\\Text_Summarization\\End_to_End_Text_Summarizer\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Projects and Work\\Projects\\NLP\\NLP PROJECTS\\Text_Summarization\\End_to_End_Text_Summarizer'

#### Creating Entities 

In [7]:
# creating entities 
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size:int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


##### Configuration Manager


In [8]:
# creating Configuration Manager 
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


In [9]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

#### Components

In [14]:
# creating components 
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PegasusTokenizerFast
from datasets import load_dataset, load_from_disk
import torch

In [20]:

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config= config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer,model= model_pegasus)

        ## Loading dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, 
        #     num_train_epochs=self.config.num_train_epochs, 
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, 
        #     per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        #     weight_decay=self.config.weight_decay,
        #     logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy,
        #       eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
            )


    

        trainer= Trainer(model= model_pegasus, 
                         args= trainer_args,
                         tokenizer= tokenizer,
                         data_collator=seq2seq_data_collector,
                         train_dataset= dataset_samsum_pt["test"],
                         eval_dataset= dataset_samsum_pt["validation"]
                         )
        trainer.train()

        ## Saving the model 
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        ## Saving tokenizer 
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

#### Pipeline 

In [12]:
## run the three commands from the main text summarizer file in the terminal 


In [21]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-13 21:58:59,386:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-09-13 21:58:59,389:INFO:common:yaml file: params.yaml loaded successfully]
[2023-09-13 21:58:59,389:INFO:common:Created directories at : artifacts]
[2023-09-13 21:58:59,390:INFO:common:Created directories at : artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Exception: 'D:\Projects and Work\Projects\NLP\NLP PROJECTS\Text_Summarization\End_to_End_Text_Summarizer\mlruns' does not exist.